In [ ]:
import itertools
import re
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome('/Users/glicia/python/chromedriver')

url = 'https://www.drugs.com/sfx/'
driver.get(url)

ses = []

#この範囲は変更しない
elem_range = driver.find_elements_by_class_name('ddc-clearfix')[1]
#Aの範囲
elem_range2 = elem_range.find_element_by_css_selector('#a-to-z > div > div:nth-child(1) > ul:nth-child(2)')

for elem in elem_range2.find_elements_by_tag_name('li'):
    driver.execute_script("arguments[0].scrollIntoView();", elem)
    sleep(0.9)
    atag1 = elem.find_element_by_tag_name('a')
    new_url = atag1.get_attribute('href')

    driver.execute_script("window.open()")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(new_url)
    sleep(0.9)
    
    #二つ目のタブ
    dg_range = driver.find_element_by_class_name('ddc-list-column-2')
    for dg_elem in dg_range.find_elements_by_tag_name('li'):
        driver.execute_script("arguments[0].scrollIntoView();", dg_elem)
        sleep(0.9)

        atag2 = dg_elem.find_element_by_tag_name('a')
        dn = atag2.text
        ltst_url = atag2.get_attribute('href')
        
        res = requests.get(ltst_url)
        soup = BeautifulSoup(res.text, 'html.parser')
        drug_name = soup.find('h1').text
        dn = drug_name.split(' Side Effects')[0]
        try:
            gntxt = soup.find('p', class_='drug-subtitle').text
            gn = gntxt.split('Generic name: ')[1]
        except:
            gn = dn
            
        try:
            soup.find('h3', text='Side effects requiring immediate medical attention').name = 'new_tag'
        except:
            pass
        
        try:
            soup.find('h3', text='Side effects not requiring immediate medical attention').name = 'new_tag'
        except:
            pass
        
        try:
            ntgs = soup.find_all('new_tag')
        except:
            pass
        
        try:
            for el in ntgs:
                els = [i for i in itertools.takewhile(lambda x : x.name not in [el.name, 'h2'], el.next_siblings)]
                border = soup.new_tag('border')
                el.wrap(border)
                for tag in els:
                    border.append(tag)
        except:
            pass
        
        #<i>と<ul>の抜き出しに成功
        se_list = []
        for j in soup.find_all('border'):
            try:
                rng_name = j.find('new_tag').text
            except:
                rng_name = 'N/A'
            for i in soup.find_all('i'):
                try:
                    if i.parent.next_sibling.next_sibling and i.parent.next_sibling.next_sibling.name == 'ul':
                        #print(i)
                        #print(i.parent.next_sibling.next_sibling)
                        fqecy = i.text
                        for se in i.parent.next_sibling.next_sibling.find_all('li'):
                            sename = se.text
                            se_list = [dn, gn, rng_name, fqecy, sename]
                            ses.append(se_list)
                except:
                    pass
                
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
driver.quit()
df = pd.DataFrame(ses, columns=['Drug_Name', 'Generic_Name', 'Requiring_Level', 'Frequency', 'Side_Effect'])
df.to_csv('dn_a_test20211220.csv', index=False)

In [4]:
#ses

In [5]:
df = pd.DataFrame(ses, columns=['Drug_Name', 'Generic_Name', 'Requiring_Level', 'Frequency', 'Side_Effect'])
df.to_csv('dn_a_test20211220.csv', index=False)